# ETL Process for Visual Analytics Exam Project



In [1]:
import pandas as pd
import countryinfo as cf # Source: https://gist.github.com/canfixit/1662664
import numpy as np

In [2]:
countries = cf.countries
eu_codes = []
eu_names = []
iso_name = {}
name_iso = {}

# Excluding european countries not relevant for the stakeholder
exclusion_list = [
    'San Marino', 'Russia', 'Vatican City', 'Monaco','Macedonia', 'Bosnia and Herzegovina', 
    'Serbia','Montenegro', 'Albania' ,'Moldova', 'Belarus', 'Andorra','Ukraine'
]
print("Exclusion list: ")
for co in countries:
    
    if co['name'] in exclusion_list:
        print(co['name'])
        continue
        
    if co['continent'].lower() == 'europe' or co['name'] == "Cyprus":
        name = co['name']
        # Chnage the name of Netherlands to a more universal used name
        if name == 'Kingdom of the Netherlands':
            name = "Netherlands"

        iso = co['code']
        eu_codes.append(iso)
        eu_names.append(name)
        iso_name[iso] = name
        name_iso[name] = iso
        
tot_num_c = len(eu_codes)

print("Number of countries: ", tot_num_c)

Exclusion list: 
Andorra
Albania
Belarus
Macedonia
Russia
San Marino
Ukraine
Vatican City
Bosnia and Herzegovina
Moldova
Monaco
Montenegro
Serbia
Number of countries:  32


In [3]:
# Main dataframe
df_main = None
# Time period from 1990 to 2022
time_period = list(range(1990, 2022))

## Temperature Offset
#### Compared to baseline beeing 1950-1980 average

In [4]:
df_temp_change = pd.read_csv("data/Annual_Surface_Temperature_Change.csv")

df_temp_change = df_temp_change[df_temp_change['ISO2'].isin(eu_codes)]

# Drop unnecessary columns and transform columns containing a year into a pure integer
# by removing the F from pre integer (e.g., F2012 -> 2012)
df_temp_change = df_temp_change.drop(['Unit','Country', 'CTS_Code', 'Indicator', 'Source', 'CTS_Name', 'ObjectId', 'CTS_Full_Descriptor'], axis = 1, inplace=False)
df_temp_change.columns = [str(x).split("F")[-1] for x in df_temp_change.columns]

# Make all columnsnames lowercase, turn the table from wide to long format, and make year-column values into integer
df_temp_change.columns= df_temp_change.columns.str.lower()
df_temp_change = pd.melt(df_temp_change, id_vars=["iso2", 'iso3'], var_name = ['year'], value_name = 'temp_change C')
df_temp_change["year"] = pd.to_numeric(df_temp_change["year"])

df_temp_change.sort_values(by=["iso3", 'year'], inplace=True)
df_temp_change.columns = ['ISO2', 'ISO3', 'year', 'Temperature Change C']

df_temp_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1952 entries, 0 to 1949
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ISO2                  1952 non-null   object 
 1   ISO3                  1952 non-null   object 
 2   year                  1952 non-null   int64  
 3   Temperature Change C  1655 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 76.2+ KB


## CO2 Emissions - Annual 
Annual CO2 emissions by country measured in tons.

In [5]:
df_co2 = pd.read_csv("data/annual-co2-emissions-per-country.csv")
df_co2.columns = df_co2.columns.str.lower()

# Rename columns and choose only contries relevant
df_co2.columns = ['country', 'ISO3', 'year', 'annual co2 emmisions']
df_co2 = df_co2[df_co2['ISO3'].isin(set(df_temp_change['ISO3']))]

# Compute the year over year change in emissions for each country
df_co2['y/y Change in Emmision (%)'] = df_co2['annual co2 emmisions'].pct_change()

df_co2["year"] = pd.to_numeric(df_co2["year"])

df_co2 = df_co2[['ISO3', 'year', 'annual co2 emmisions', 'y/y Change in Emmision (%)']]
df_co2.rename(columns={'annual co2 emmisions' : 'annual t co2 emmisions'}, inplace = True)

df_co2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5253 entries, 2563 to 29726
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ISO3                        5253 non-null   object 
 1   year                        5253 non-null   int64  
 2   annual t co2 emmisions      5253 non-null   float64
 3   y/y Change in Emmision (%)  5066 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 205.2+ KB


## GDP
Measured in US Dollar

In [6]:
df_gdp = pd.read_csv("data/GDP.csv",delimiter=";")

#Drop last column, as it is no values
df_gdp = df_gdp.iloc[: , :-1]

df_gdp = df_gdp[df_gdp['Country Code'].isin(set(df_temp_change['ISO3']))]

# Drop unrelevant columns and rename to ISO3 where code is
df_gdp.drop(['Country Name', 'Indicator Name', 'Indicator Code'], axis = 1, inplace = True)
df_gdp.columns = df_gdp.columns.str.lower()
df_gdp.rename(columns = {'country code' : "ISO3"}, inplace=True)

# Melt from wide to long format
df_gdp = pd.melt(df_gdp, id_vars=["ISO3"], var_name = ['year'], value_name = 'GDP US$')
df_gdp["year"] = pd.to_numeric(df_gdp["year"])

df_gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1984 entries, 0 to 1983
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ISO3     1984 non-null   object 
 1   year     1984 non-null   int64  
 2   GDP US$  1578 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 46.6+ KB


## Green Bonds Annual Issued
Initially measured in Billion US Dollars.

In [7]:
df_green_bonds = pd.read_csv("data/Green_Bonds.csv")

# Choose only the relevant countries
df_green_bonds = df_green_bonds[df_green_bonds['ISO3'].isin(set(df_temp_change['ISO3']))]

# Drop unrelevant columns and fix column names from e.g., F2012 to 2012
df_green_bonds.drop(['ISO2', 'Indicator', 'Source', 'CTS_Code', 'CTS_Full_Descriptor', 'Type_of_Issuer', 'Use_of_Proceed', 'Principal_Currency'],axis = 1, inplace=True)
df_green_bonds.columns = df_green_bonds.columns.str.lower()
df_green_bonds.columns = [str(x).split("f")[-1] for x in df_green_bonds.columns]

# Only keep cts_name equals Green Bonds Issuances, as we can calculate total later (cts_name == Green Bonds)
df_green_bonds = df_green_bonds.loc[df_green_bonds['cts_name'] == 'Green Bonds Issuances'] 
df_green_bonds.drop(['cts_name', 'unit', 'objectid', 'country'],axis = 1, inplace=True)

df_green_bonds.rename(columns = {'iso3' : "ISO3"}, inplace=True)

# Convert from wide to long format
df_green_bonds = pd.melt(df_green_bonds, id_vars=["ISO3"], var_name = ['year'], value_name = 'Green Bonds Issuance (US$)')
df_green_bonds["year"] = pd.to_numeric(df_green_bonds["year"])
df_green_bonds.sort_values(by=["ISO3", 'year'], inplace=True)

# Compute year over year change in green bonds
df_green_bonds['Change Green Bonds'] = df_green_bonds['Green Bonds Issuance (US$)'].pct_change()


# Make Green Bonds in whole numbers, not billion
df_green_bonds['Green Bonds Issuance (US$)'] = df_green_bonds['Green Bonds Issuance (US$)'] * 1_000_000_000

df_green_bonds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 784 entries, 0 to 781
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ISO3                        784 non-null    object 
 1   year                        784 non-null    int64  
 2   Green Bonds Issuance (US$)  156 non-null    float64
 3   Change Green Bonds          749 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 30.6+ KB


As we observe from the output, we are missing substantial data from some countries, as well as from many years, as green bonds issuance is quite new

## Population

In [8]:
df_pop = pd.read_csv("data/population.csv")

df_pop = df_pop[df_pop['Country Code'].isin(set(df_temp_change['ISO3']))]
df_pop.columns = df_pop.columns.str.lower()

# Fix column names as they contained "[]" in some instances
df_pop.columns = [str(x).split("[")[0] for x in df_pop.columns]
df_pop.drop(['series name', 'series code', 'country name'] ,axis=1, inplace = True)
df_pop.rename(columns = {'country code' : "ISO3"}, inplace=True)

# Convert table from wide to long format and change the data types for year and population
df_pop = pd.melt(df_pop, id_vars=["ISO3"], var_name = ['year'], value_name = 'population')
df_pop["year"] = pd.to_numeric(df_pop["year"])
df_pop["population"] = pd.to_numeric(df_pop["population"])

df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ISO3        1600 non-null   object
 1   year        1600 non-null   int64 
 2   population  1600 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 37.6+ KB


## Electric Vehicle Sales as fraction of all Personal Vehicle Sales

BEVs are battery electric vehicles. 

PHEVs are plug-in hybrid electric vehicles. 

EVs refers to all electric vehicles (BEVs + PHEVs).

In [9]:
df_ev_share = pd.read_csv("data/IEA-EV-dataEV sales shareCarsHistorical.csv")
df_ev_share = df_ev_share[['region', 'year', 'value']]

# Make all values in percent (*0.01)
df_ev_share['value'] = df_ev_share['value'] * 0.01
df_ev_share = df_ev_share[df_ev_share['region'].isin(eu_names)]

# Replace all country names with their respective ISO2 code from the countryinfo dataset
# for compitability when merging all datasets
df_ev_share['region'].replace(name_iso, inplace = True)
df_ev_share.rename(columns = {"region" : "ISO2", "value" : "EV sales share"}, inplace = True)

df_ev_share.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 11 to 332
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ISO2            187 non-null    object 
 1   year            187 non-null    int64  
 2   EV sales share  187 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 5.8+ KB


# Merging into one main dataframe
- With average and total as two seperate datasheets

In [10]:
# Initiate the main dataframe as temperature change df as this dataframe contains all ISO2 and ISO3 codes
df_main = df_temp_change

# Check if all conseccutive years are present
# Check if all years still in the dataset for every country

for c in df_main['ISO3'].unique():
    for_this_c = df_main[df_main['ISO3'] == c]
    all_years = (for_this_c['year'] == list(range(min(for_this_c['year']), max(for_this_c['year']) + 1))).all()
    if not all_years:
        print("Not Consecutive years included for: ", c)

# Check if all countries in dataset
print("All countries included before merge: ",len(df_main['ISO2'].unique()) == len(eu_codes))

# Merge all dataframes

dfs = [df_co2, df_gdp, df_green_bonds, df_pop]

for frame in dfs:
    
    df_main = df_main.merge(frame, left_on=['ISO3','year'], right_on = ['ISO3', 'year'], how = 'left')

# Merge EV-files on ISO2
dfs = [df_ev_share]
for frame in dfs:
    df_main = df_main.merge(frame, left_on=['ISO2','year'], right_on = ['ISO2', 'year'], how = 'left')
    
# Insert country name
df_main['country'] = df_main['ISO2']
df_main['country'].replace(iso_name, inplace=True)


# Check if all countries still in the dataset
print("All countries included after merge: ",len(df_main['country'].unique()) == len(eu_codes))

# Check if all countries still have consecutive years in the dataset
for c in df_main['ISO3'].unique():
    for_this_c = df_main[df_main['ISO3'] == c]
    all_years = (for_this_c['year'] == list(range(min(for_this_c['year']), max(for_this_c['year']) + 1))).all()
    if not all_years:
        print("Not Consecutive years included for: ", c)

# REpresenting the DF for one country
df_main[df_main['ISO3'] == 'NOR']

All countries included before merge:  True
All countries included after merge:  True


,ISO2,ISO3,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (US$),Change Green Bonds,population,EV sales share,country
1525,NO,NOR,1961,1.242,13335483.0,0.019002,5.632461e+09,NaN,NaN,NaN,NaN,Norway
1526,NO,NOR,1962,-0.779,14060391.0,0.054359,6.066977e+09,NaN,NaN,NaN,NaN,Norway
1527,NO,NOR,1963,-0.137,14965280.0,0.064357,6.510240e+09,NaN,NaN,NaN,NaN,Norway
1528,NO,NOR,1964,0.365,16170321.0,0.080522,7.159203e+09,NaN,NaN,NaN,NaN,Norway
1529,NO,NOR,1965,-0.269,16389921.0,0.013580,8.058681e+09,NaN,NaN,NaN,NaN,Norway
...,...,...,...,...,...,...,...,...,...,...,...,...
1581,NO,NOR,2017,1.578,44242000.0,-0.011683,3.983940e+11,6.870686e+08,-0.393913,5276968.0,0.39,Norway
1582,NO,NOR,2018,1.510,44393000.0,0.003413,4.369997e+11,3.318214e+09,3.829524,5311916.0,0.49,Norway
1583,NO,NOR,2019,1.231,42784000.0,-0.036244,4.049414e+11,5.877162e+09,0.771182,5347896.0,0.56,Norway
1584,NO,NOR,2020,2.421,41196000.0,-0.037117,3.621983e+11,6.786820e+09,0.154778,5379475.0,0.75,Norway


In [11]:
df_main.describe()

,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (US$),Change Green Bonds,population,EV sales share
count,1952.000000,1655.000000,1.923000e+03,1923.000000,1.562000e+03,1.560000e+02,749.0,1.600000e+03,187.000000
mean,1991.000000,0.733279,1.292379e+08,0.011215,3.408779e+11,5.539923e+09,inf,1.552739e+07,0.065630
std,17.611329,0.871814,2.013136e+08,0.073343,6.545307e+11,1.036716e+10,NaN,2.099156e+07,0.136073
min,1961.000000,-1.793000,1.419960e+05,-0.972617,9.009833e+07,0.000000e+00,-1.0,2.216100e+04,0.000013
25%,1976.000000,0.044000,1.739022e+07,-0.026693,2.001223e+10,4.161317e+08,0.0,2.762342e+06,0.001650
50%,1991.000000,0.753000,4.958335e+07,0.010317,8.503060e+10,1.571946e+09,0.0,7.544770e+06,0.010000
75%,2006.000000,1.379500,1.258108e+08,0.049286,2.988262e+11,5.333590e+09,0.0,1.202306e+07,0.055000
max,2021.000000,3.595000,1.117888e+09,0.367647,4.223116e+12,7.170096e+10,inf,8.316087e+07,0.860000


In [12]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1952 entries, 0 to 1951
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ISO2                        1952 non-null   object 
 1   ISO3                        1952 non-null   object 
 2   year                        1952 non-null   int64  
 3   Temperature Change C        1655 non-null   float64
 4   annual t co2 emmisions      1923 non-null   float64
 5   y/y Change in Emmision (%)  1923 non-null   float64
 6   GDP US$                     1562 non-null   float64
 7   Green Bonds Issuance (US$)  156 non-null    float64
 8   Change Green Bonds          749 non-null    float64
 9   population                  1600 non-null   float64
 10  EV sales share              187 non-null    float64
 11  country                     1952 non-null   object 
dtypes: float64(8), int64(1), object(3)
memory usage: 198.2+ KB


## Average DataFrame

As the analytical tool (Tableau) handles filters badly with calculation of average when only showcasing one value from the dataset.

In [13]:
# Making a average DataFrame
df_average = df_main.groupby(["year"], as_index=False)
df_average = df_average.mean()

# Make change in percentage points for EV sales share as this year - last year
# since the data is represented in percent, it makes sense to do it this way
df_average['ev sales share change'] = df_average['EV sales share'].diff()
df_average


,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (US$),Change Green Bonds,population,EV sales share,ev sales share change
0,1961,0.907957,9.104080e+07,0.039306,1.799943e+10,NaN,NaN,NaN,NaN,NaN
1,1962,-0.219217,9.632505e+07,0.075410,1.963547e+10,NaN,NaN,NaN,NaN,NaN
2,1963,-0.481087,1.019603e+08,0.088141,2.166625e+10,NaN,NaN,NaN,NaN,NaN
3,1964,-0.044739,1.063450e+08,0.073750,2.398202e+10,NaN,NaN,NaN,NaN,NaN
4,1965,-0.516870,1.079438e+08,0.042073,2.612632e+10,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
56,2017,1.512188,1.123991e+08,0.017202,5.812301e+11,4.398360e+09,0.750004,1.644860e+07,0.047112,0.012949
57,2018,2.005688,1.098124e+08,-0.014684,6.276566e+11,4.610240e+09,9.726477,1.648726e+07,0.064950,0.017838
58,2019,1.936500,1.048245e+08,-0.036983,6.168287e+11,8.428476e+09,1.952621,1.651112e+07,0.088994,0.024044
59,2020,2.281813,9.459273e+07,-0.088455,6.191357e+11,8.403601e+09,0.099287,1.653066e+07,0.191025,0.102031


## Total DataFrame

In [14]:

df_total = df_main.groupby(["year"], as_index=False)
df_total = df_total.sum()

# Calculating percentage change YoY
df_total['y/y Change in Emmision (%)'] = df_total['annual t co2 emmisions'].pct_change()
df_total['Change Green Bonds'] = df_total['Green Bonds Issuance (US$)'].pct_change()
df_total['Change Green Bonds'] = df_total['Change Green Bonds'].astype(float)

df_total.head()

,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (US$),Change Green Bonds,population,EV sales share
0,1961,20.883,2.822265e+09,NaN,2.879909e+11,0.0,NaN,0.0,0.0
1,1962,-5.042,2.986077e+09,0.058043,3.141676e+11,0.0,NaN,0.0,0.0
2,1963,-11.065,3.160770e+09,0.058503,3.466600e+11,0.0,NaN,0.0,0.0
3,1964,-1.029,3.296694e+09,0.043003,3.837123e+11,0.0,NaN,0.0,0.0
4,1965,-11.888,3.346259e+09,0.015035,4.180211e+11,0.0,NaN,0.0,0.0


In [15]:
# Make year values into datetime format from integer as well as only keeping
# the wanted time period in the dataset, as initiated at the start of this workbook

df_main = df_main[df_main['year'].isin(set(time_period))]
df_main = df_main.astype({'year' : 'int32'}, copy = True)
df_main.year = pd.to_datetime(df_main.year, format='%Y')

df_average = df_average[df_average['year'].isin(set(time_period))]
df_average = df_average.astype({'year' : 'int32'}, copy = True)
df_average.year = pd.to_datetime(df_average.year, format='%Y')

df_total  = df_total[df_total['year'].isin(set(time_period))]
df_total = df_total.astype({'year' : 'int32'}, copy = True)
df_total.year = pd.to_datetime(df_total.year, format='%Y')


In [16]:
# Construct change since 1990 for all countries annual emission
dfs = df_main.groupby(by="ISO3")
firsts = dfs['annual t co2 emmisions'].first()
    
# Calculating emission change since 1990 for all countries in dataset
ds = df_main.groupby('ISO3')['annual t co2 emmisions'].apply(lambda c: c.div(c.iloc[0]).subtract(1))
df_main['emission change since 1990'] = ds

# Test to see if calculation above is accurate
first = df_main[df_main['ISO3'] == "NOR"]['annual t co2 emmisions'].tolist()[0]
rest = np.array(df_main[df_main['ISO3'] == "NOR"]['annual t co2 emmisions'].tolist())

test = (rest-first)/first
calc = np.array(df_main[df_main['ISO3'] == "NOR"]['emission change since 1990'].tolist())

test = [round(c, 5) for c in test]
calc = [round(c, 5) for c in calc]

print("Calculation valid: ", test == calc)
df_main[df_main['ISO3'] == "NOR"].head(5)

Calculation valid:  True


,ISO2,ISO3,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (US$),Change Green Bonds,population,EV sales share,country,emission change since 1990
1554,NO,NOR,1990-01-01,1.532,35097000.0,0.025212,1.197917e+11,NaN,0.0,4241473.0,NaN,Norway,0.000000
1555,NO,NOR,1991-01-01,0.992,33535000.0,-0.044505,1.218725e+11,NaN,0.0,4261732.0,NaN,Norway,-0.044505
1556,NO,NOR,1992-01-01,1.134,34455000.0,0.027434,1.308380e+11,NaN,0.0,4286401.0,NaN,Norway,-0.018292
1557,NO,NOR,1993-01-01,0.712,36000000.0,0.044841,1.205791e+11,NaN,0.0,4311991.0,NaN,Norway,0.025729
1558,NO,NOR,1994-01-01,0.081,37905000.0,0.052917,1.271315e+11,NaN,0.0,4336613.0,NaN,Norway,0.080007


In [17]:
# Compute Emission Change from 1990 to all years, for the total dataframe
first_value = df_total['annual t co2 emmisions'].tolist()[0]
df_total['emission change since 1990'] = (df_total['annual t co2 emmisions'] - first_value)/first_value
df_total

,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (US$),Change Green Bonds,population,EV sales share,emission change since 1990
29,1990-01-01,27.166,4.549109e+09,0.012855,7.903037e+12,0.000000e+00,NaN,488966193.0,0.000000,0.000000
30,1991-01-01,7.386,4.494420e+09,-0.012022,8.227680e+12,2.917930e+07,inf,490504090.0,0.000000,-0.012022
31,1992-01-01,21.732,4.351142e+09,-0.031879,8.950102e+12,1.435457e+08,3.919436,491996666.0,0.000000,-0.043518
32,1993-01-01,7.359,4.268875e+09,-0.018907,8.172726e+12,1.856502e+08,0.293318,493603706.0,0.000000,-0.061602
33,1994-01-01,34.283,4.245703e+09,-0.005428,8.685738e+12,1.097454e+08,-0.408859,494891715.0,0.000000,-0.066696
34,1995-01-01,28.153,4.287476e+09,0.009839,1.015716e+13,0.000000e+00,-1.000000,495920609.0,0.000000,-0.057513
35,1996-01-01,-4.137,4.395844e+09,0.025275,1.036695e+13,0.000000e+00,NaN,496817267.0,0.000000,-0.033691
36,1997-01-01,17.012,4.305054e+09,-0.020654,9.759677e+12,0.000000e+00,NaN,497652023.0,0.000000,-0.053649
37,1998-01-01,26.578,4.303590e+09,-0.000340,1.009221e+13,0.000000e+00,NaN,498444826.0,0.000000,-0.053971
38,1999-01-01,31.208,4.242333e+09,-0.014234,1.008396e+13,4.817370e+07,inf,499414002.0,0.000000,-0.067436


In [18]:
## Write all DataFrames to excel

writer = pd.ExcelWriter('data/main.xlsx', engine='xlsxwriter')

df_main.to_excel(writer, sheet_name = "main")
df_average.to_excel(writer, sheet_name = "average")
df_total.to_excel(writer, sheet_name = "total")

writer.save()